# 40 · Supply-Chain Generation Demo — *“What goes into an Electric Vehicle?”*  
_Last updated 2025-05-03_

We’ll use GPT-4o-mini to **generate** a bill-of-inputs for manufacturing an electric car:

1. **Prompt** the model as a supply-chain analyst.  
2. **Constrain** output via a strict JSON schema (`products[]`).  
3. **Parse → DataFrame → Quick sanity check**.



### If you use this code, please cite the paper: 

- Fetzer, T., Lambert, P.J., Feld, B. and Garg, P., 2024. **AI-generated production networks: Measurement and applications to global trade**.


### API key

* Looks for `OPENAI_API_KEY` in the environment.  
* Else tries `key/openai_key.txt` (single-line file).  
* Raises a clear error if neither is found.

Same helper pattern as earlier notebooks → students configure once, reuse everywhere.


In [1]:
# %pip -q install --upgrade openai

import os, pathlib, json, pandas as pd
from openai import OpenAI

# --- locate key ------------------------------------------------------
key_path = pathlib.Path("key/openai_key.txt")
if os.getenv("OPENAI_API_KEY") is None and key_path.exists():
    os.environ["OPENAI_API_KEY"] = key_path.read_text().strip()

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "No API key found.  Place it in key/openai_key.txt or export OPENAI_API_KEY."
    )

client = OpenAI()


### Response schema (`supply_chain_response`)

| field | type | description |
|-------|------|-------------|
| `product_description` | string | HS-style description (“Lithium-ion battery pack”) |
| `importance_score` (1-10) | integer | 10 = mission-critical, 1 = minor |
| `classification` | `"intermediary"` or `"capital"` | Working input vs. capital equipment |

Why strict?  
*Downstream dashboards don’t break if the model adds creative keys.*


In [2]:
schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "supply_chain_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "products": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "product_description": {"type": "string"},
                            "importance_score":   {"type": "integer"},
                            "classification":     {
                                "type": "string",
                                "enum": ["intermediary", "capital"]
                            }
                        },
                        "required": [
                            "product_description",
                            "importance_score",
                            "classification"
                        ],
                        "additionalProperties": False
                    }
                }
            },
            "required": ["products"],
            "additionalProperties": False
        }
    }
}

prompt = (
    "You are a supply-chain analyst.\n"
    "List **up to 10** input goods (intermediary or capital) essential for "
    "producing a modern **electric vehicle**.\n"
    "Return JSON conforming exactly to the schema."
)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": prompt}],
    temperature=0.7,
    max_tokens=800,
    response_format=schema
)

products = json.loads(response.choices[0].message.content)["products"]
df = pd.DataFrame(products)
df


,product_description,importance_score,classification
0,Lithium-ion battery cells,10,intermediary
1,Electric motor,9,intermediary
2,Power electronics,8,intermediary
3,Vehicle chassis,7,capital
4,Battery management system,8,intermediary
5,Charging infrastructure,6,capital
6,Electric drivetrain components,8,intermediary
7,Steel and aluminum for body construction,7,capital
8,Software for vehicle control systems,7,intermediary
9,Thermal management systems,6,intermediary


### Quick sanity check

* **Core components present?**  (Li-ion battery, power electronics, electric motor…)  
* **Any surprises?**  (Did it include something debatable like “steel sheet”?)  
* **Score distribution** — do the 9-10 scores align with your intuition?

If you spot a *missing* critical input (e.g. “rare-earth magnets”), mention it:
> “Models sometimes omit niche but vital items — refine the prompt or run a second pass.”


**Exercise ideas**

1. **Swap the product** — run the notebook for “smartphone” or “solar panel”.  
2. **Change temperature** — see how creative vs. strict outputs differ.  
3. **Add cost share** — extend the schema with a `cost_share_pct` field and ask the
   model to estimate it (then discuss reliability).  
4. **Map to HS-6 codes** — feed each `product_description` through the
   embedding-matching notebook (31) to align with trade data.
